# Modelos

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Wine_reviews_climate_prediction.csv to Wine_reviews_climate_prediction.csv
User uploaded file "Wine_reviews_climate_prediction.csv" with length 29206594 bytes


## 1. Cargue de Librerias, base de datos y entrenamiento.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error,r2_score,mean_absolute_error,mean_squared_error

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from scipy.stats import uniform

from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeRegressor

from sklearn.svm import SVC, SVR, LinearSVR

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('Wine_reviews_climate_prediction.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70030 entries, 0 to 70029
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      70030 non-null  int64  
 1   country         70030 non-null  object 
 2   description     70030 non-null  object 
 3   points          70030 non-null  int64  
 4   price           70030 non-null  float64
 5   taster_name     70030 non-null  object 
 6   variety         70030 non-null  object 
 7   winery          70030 non-null  object 
 8   Year            70030 non-null  int64  
 9   region          70030 non-null  object 
 10  Latitude        70030 non-null  float64
 11  Longitude       70030 non-null  float64
 12  Lat_x           70030 non-null  float64
 13  Long_x          70030 non-null  float64
 14  temp_anual      70030 non-null  float64
 15  temp_max_anual  70030 non-null  float64
 16  temp_min_anual  70030 non-null  float64
 17  pre_anual       70030 non-null 

In [6]:
df.isnull().sum()

Unnamed: 0        0
country           0
description       0
points            0
price             0
taster_name       0
variety           0
winery            0
Year              0
region            0
Latitude          0
Longitude         0
Lat_x             0
Long_x            0
temp_anual        0
temp_max_anual    0
temp_min_anual    0
pre_anual         0
etp_anual         0
dtype: int64

In [ ]:
#Deleteing rows with price = 0
df.dropna(subset = ['price'], inplace=True)
df.isnull().sum()

Unnamed: 0        0
country           0
description       0
points            0
price             0
taster_name       0
variety           0
winery            0
Year              0
region            0
Latitude          0
Longitude         0
Lat_x             0
Long_x            0
temp_anual        0
temp_max_anual    0
temp_min_anual    0
pre_anual         0
etp_anual         0
dtype: int64

In [7]:
df.drop(columns=['description', 'variety', 'winery', 'Latitude', 'Longitude', 'region', 'Unnamed: 0'], inplace=True)
df

,country,points,price,taster_name,Year,Lat_x,Long_x,temp_anual,temp_max_anual,temp_min_anual,pre_anual,etp_anual
0,Portugal,87,15.0,Roger Voss,2011,41.75,-5.75,13.01,19.44,6.61,388.5,1200
1,Portugal,87,15.0,Roger Voss,2011,41.75,-5.75,13.01,19.44,6.61,388.5,1200
2,Portugal,87,17.0,Roger Voss,2011,41.75,-5.75,13.01,19.44,6.61,388.5,1200
3,Portugal,91,12.0,Roger Voss,2011,41.75,-5.75,13.01,19.44,6.61,388.5,1200
4,Portugal,87,8.0,Roger Voss,2011,41.75,-5.75,13.01,19.44,6.61,388.5,1200
...,...,...,...,...,...,...,...,...,...,...,...,...
70025,USA,84,25.0,Matt Kettmann,2012,34.75,-118.25,13.96,20.23,7.75,325.0,1176
70026,Switzerland,90,21.0,Jeff Jenssen,2012,46.75,6.75,9.15,13.46,4.87,1382.5,696
70027,France,89,14.0,Roger Voss,2012,45.25,6.25,5.61,9.20,2.05,1386.2,714
70028,France,89,18.0,Roger Voss,2012,45.75,5.75,10.71,15.25,6.22,976.2,807


In [8]:
df = pd.get_dummies(df, columns=['country', 'taster_name'])

In [9]:
data = df

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70030 entries, 0 to 70029
Data columns (total 70 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   points                          70030 non-null  int64  
 1   price                           70030 non-null  float64
 2   Year                            70030 non-null  int64  
 3   Lat_x                           70030 non-null  float64
 4   Long_x                          70030 non-null  float64
 5   temp_anual                      70030 non-null  float64
 6   temp_max_anual                  70030 non-null  float64
 7   temp_min_anual                  70030 non-null  float64
 8   pre_anual                       70030 non-null  float64
 9   etp_anual                       70030 non-null  int64  
 10  country_Argentina               70030 non-null  uint8  
 11  country_Australia               70030 non-null  uint8  
 12  country_Austria                 

In [11]:
#First we separate our info, the X will be all the columns except the output variable, then the output variable, the test size 20% and the random state to have always the same output
# is like X, X_test, y, y_test = train_test_split(variables_X, variable_Y, test_size, random_state)
X, X_test, y, y_test = train_test_split(data.drop(columns='points'), data.points, test_size=0.2, random_state=42)

## Modelo 1: *Ridge con Polynomial Features*

In [ ]:
%%time
#We create the Pipeline. First step, Normalize data, include the Polynomimal 
#(include bias false to remove from the Polynomial for X[0]=1) and then use the Ridge method
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('ridge', Ridge())
])

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
dists = {'ridge__alpha' : uniform(loc=0, scale=200), 'poly__degree' : [1, 2]}
#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
ridge_CV = RandomizedSearchCV(ridge_pipe, param_distributions=dists, n_iter=30, cv=5)
#Now we train the model
ridge_CV.fit(X, y)

CPU times: user 22min 15s, sys: 1min 21s, total: 23min 36s
Wall time: 12min 44s


In [ ]:
ridge_CV.best_params_

{'poly__degree': 2, 'ridge__alpha': 99.8182086475239}

In [ ]:
ridge_CV.score(X, y)

0.3825488102240244

In [ ]:
ridge_CV.score(X_test, y_test)

0.37879961343187485

In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,ridge_CV.predict(X)))

Error absoluto medio (MAE): 1.9608568194536513
Error Cuadrático Medio (MSE): 6.131192514257583
Raíz del Error Cuadrático Medio (RMSE): 2.476124494902787
Coeficiente de determinación R2: -0.6392490725811766


In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,ridge_CV.predict(X_test)))

Error absoluto medio (MAE): 1.9834422010623907
Error Cuadrático Medio (MSE): 6.188017041504844
Raíz del Error Cuadrático Medio (RMSE): 2.487572519848385
Coeficiente de determinación R2: -0.7039695549971567


## Modelo 2: *Arbol de decisión*

In [ ]:
%%time
#Now we try a Tree Regressor
tree = DecisionTreeRegressor()

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
param_grid = {'max_depth' : [10, 11, 12, 13, 14, 15], 'min_samples_leaf' : [47, 48, 49, 50, 51, 52, 53]}

#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
tree_CV = GridSearchCV(tree, param_grid=param_grid)
#Now we train the model
tree_CV.fit(X, y)

CPU times: user 59.5 s, sys: 322 ms, total: 59.8 s
Wall time: 59.7 s


In [ ]:
print("The best Parameters are :", tree_CV.best_params_)
print("The best X,y score is: ", tree_CV.score(X, y))
print("The best X,y test score is: ", tree_CV.score(X_test, y_test))

The best Parameters are : {'max_depth': 11, 'min_samples_leaf': 48}
The best X,y score is:  0.4729605577155309
The best X,y test score is:  0.43547837376177856


In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(tree_CV.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(tree_CV.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(tree_CV.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,tree_CV.predict(X)))

Error absoluto medio (MAE): 1.8068385026623235
Error Cuadrático Medio (MSE): 5.233418182294606
Raíz del Error Cuadrático Medio (RMSE): 2.2876665365158897
Coeficiente de determinación R2: 0.4729605577155309


In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(tree_CV.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(tree_CV.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(tree_CV.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,tree_CV.predict(X_test)))

Error absoluto medio (MAE): 1.8706502617401
Error Cuadrático Medio (MSE): 5.623418012920129
Raíz del Error Cuadrático Medio (RMSE): 2.371374709513477
Coeficiente de determinación R2: 0.43547837376177856


## Modelo 3: *PCA + Ridge*

In [ ]:
%%time
#We create the Pipeline. First step, Normalize data, include the Polynomimal 
#(include bias false to remove from the Polynomial for X[0]=1) and then use the Ridge method
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('ridge', Ridge())
])

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
dists = {'ridge__alpha' : uniform(loc=0, scale=200), 'pca__n_components' : [10, 20, 30, 40, 60]}
#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
ridge_CV = RandomizedSearchCV(ridge_pipe, param_distributions=dists, n_iter=30, cv=5)
#Now we train the model
ridge_CV.fit(X, y)

CPU times: user 1min 39s, sys: 1min 15s, total: 2min 54s
Wall time: 1min 29s


In [ ]:
ridge_CV.best_params_

{'pca__n_components': 60, 'ridge__alpha': 181.33453634789183}

In [ ]:
ridge_CV.score(X, y)

0.2548815500808175

In [ ]:
ridge_CV.score(X_test, y_test)

0.2760338393516947

In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,ridge_CV.predict(X)))

Error absoluto medio (MAE): 2.1527988077283338
Error Cuadrático Medio (MSE): 7.398908185823144
Raíz del Error Cuadrático Medio (RMSE): 2.7200934149075
Coeficiente de determinación R2: -1.9396454897372628


In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,ridge_CV.predict(X_test)))

Error absoluto medio (MAE): 2.1635052958599075
Error Cuadrático Medio (MSE): 7.211706619041597
Raíz del Error Cuadrático Medio (RMSE): 2.6854620866885455
Coeficiente de determinación R2: -2.5206655084528107


**Se repite el modelo con nuevos parametros**

In [ ]:
%%time
#We create the Pipeline. First step, Normalize data, include the Polynomimal 
#(include bias false to remove from the Polynomial for X[0]=1) and then use the Ridge method
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('ridge', Ridge())
])

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
dists = {'ridge__alpha' : uniform(loc=0, scale=200), 'pca__n_components' : [60, 62, 64, 66]}
#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
ridge_CV = RandomizedSearchCV(ridge_pipe, param_distributions=dists, n_iter=30, cv=5)
#Now we train the model
ridge_CV.fit(X, y)

CPU times: user 1min 23s, sys: 27.4 s, total: 1min 50s
Wall time: 58 s


In [ ]:
ridge_CV.best_params_

{'pca__n_components': 66, 'ridge__alpha': 150.3017831570962}

In [ ]:
ridge_CV.score(X, y)

0.25563303886045785

In [ ]:
ridge_CV.score(X_test, y_test)

0.27637978309300126

In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,ridge_CV.predict(X)))

Error absoluto medio (MAE): 2.1512349713500676
Error Cuadrático Medio (MSE): 7.391446021272208
Raíz del Error Cuadrático Medio (RMSE): 2.7187213945662414
Coeficiente de determinación R2: 0.25563303886045785


In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,ridge_CV.predict(X_test)))

Error absoluto medio (MAE): 2.162978655305005
Error Cuadrático Medio (MSE): 7.208260539784574
Raíz del Error Cuadrático Medio (RMSE): 2.6848203924628877
Coeficiente de determinación R2: 0.27637978309300126


## Modelo 4: *PCA + Ridge + Polynomial Features*

In [ ]:
%%time
#We create the Pipeline. First step, Normalize data, include the Polynomimal 
#(include bias false to remove from the Polynomial for X[0]=1) and then use the Ridge method
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('pca', PCA()),
    ('ridge', Ridge())
])

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
dists = {'ridge__alpha' : uniform(loc=0, scale=200), 'poly__degree': [1,2], 'pca__n_components' : [30, 40, 50, 60]}
#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
ridge_CV = RandomizedSearchCV(ridge_pipe, param_distributions=dists, n_iter=30, cv=5)
#Now we train the model
ridge_CV.fit(X, y)

Wall time: 5min 44s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('poly',
                                              PolynomialFeatures(include_bias=False)),
                                             ('pca', PCA()),
                                             ('ridge', Ridge())]),
                   n_iter=30,
                   param_distributions={'pca__n_components': [30, 40, 50, 60],
                                        'poly__degree': [1, 2],
                                        'ridge__alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001BF44E5AF40>})

In [ ]:
print("The best Parameters are :", ridge_CV.best_params_)
print("The best X,y score is: ", ridge_CV.score(X, y))
print("The best X,y test score is: ", ridge_CV.score(X_test, y_test))
print("The MAE X,y is: ", mean_absolute_error(ridge_CV.predict(X), y))
print("The MAE test X,y is: ", mean_absolute_error(ridge_CV.predict(X_test), y_test))

The best Parameters are : {'pca__n_components': 60, 'poly__degree': 1, 'ridge__alpha': 172.52565962730432}
The best X,y score is:  0.2548817850785814
The best X,y test score is:  0.27604559239218884
The MAE X,y is:  2.152768892577137
The MAE test X,y is:  2.1634719034809025


In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(ridge_CV.predict(X), y))

In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(ridge_CV.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(ridge_CV.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(ridge_CV.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(ridge_CV.predict(X_test), y_test))

## Modelo 5: *Regresión lineal*

In [ ]:
%%time
#We create the Pipeline. First step, Normalize data, include the Polynomimal 
#(include bias false to remove from the Polynomial for X[0]=1) and then use the Ridge method
linear_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear', LinearRegression())
])

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
#dists = {'ridge__alpha' : uniform(loc=0, scale=200), 'poly__degree': [1,2], 'pca__n_components' : [30, 40, 50, 60]}
#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
#Now we train the model
linear_pipe.fit(X, y)

CPU times: user 370 ms, sys: 26 ms, total: 396 ms
Wall time: 262 ms


In [ ]:
print("The best X,y score is: ", linear_pipe.score(X, y))
print("The best X,y test score is: ", linear_pipe.score(X_test, y_test))
print("The MAE X,y is: ", mean_absolute_error(linear_pipe.predict(X), y))
print("The MAE test X,y is: ", mean_absolute_error(linear_pipe.predict(X_test), y_test))

The best X,y score is:  0.2562764691141056
The best X,y test score is:  -5.6815694609358496e+19
The MAE X,y is:  2.1501482834263026
The MAE test X,y is:  201018947.98970672


In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(linear_pipe.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(linear_pipe.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(linear_pipe.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,linear_pipe.predict(X)))

Error absoluto medio (MAE): 2.1501482834263026
Error Cuadrático Medio (MSE): 7.385056860768617
Raíz del Error Cuadrático Medio (RMSE): 2.717546110145809
Coeficiente de determinación R2: 0.2562764691141056


In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(linear_pipe.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(linear_pipe.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(linear_pipe.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,linear_pipe.predict(X_test)))

Error absoluto medio (MAE): 201018947.98970672
Error Cuadrático Medio (MSE): 5.659630838447473e+20
Raíz del Error Cuadrático Medio (RMSE): 23789978643.217552
Coeficiente de determinación R2: -5.6815694609358496e+19


## Modelo 6: *Support Vector Machines, SVM*

In [ ]:
%%time
svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', LinearSVR(C=1000, epsilon=1, max_iter=10000, random_state=42))
])

svm.fit(X, y)

CPU times: user 3min 59s, sys: 131 ms, total: 3min 59s
Wall time: 3min 58s


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
print("The best X,y score is: ", svm.score(X, y))
print("The best X,y test score is: ", svm.score(X_test, y_test))
print("The MAE X,y is: ", mean_absolute_error(svm.predict(X), y))
print("The MAE test X,y is: ", mean_absolute_error(svm.predict(X_test), y_test))

The best X,y score is:  0.03367518705837347
The best X,y test score is:  0.14272707474426827
The MAE X,y is:  2.2704148752770594
The MAE test X,y is:  2.2635366707544993


In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(svm.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(svm.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(svm.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,svm.predict(X)))

In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(svm.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(svm.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(svm.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,svm.predict(X_test)))

## Modelo 7: *Support Vector Machines, SVM con RBF Kernel*

In [ ]:
%%time
svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVR(kernel='rbf', C=1000, epsilon=0.5))
])

svm.fit(X, y)

Wall time: 22min 10s


Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVR(C=1000, epsilon=0.5))])

In [ ]:
print("The best X,y score is: ", svm.score(X, y))
print("The best X,y test score is: ", svm.score(X_test, y_test))
print("The MAE X,y is: ", mean_absolute_error(svm.predict(X), y))
print("The MAE test X,y is: ", mean_absolute_error(svm.predict(X_test), y_test))

The best X,y score is:  0.48239334737287676
The best X,y test score is:  0.4509594337442103
The MAE X,y is:  1.7598939589719105
The MAE test X,y is:  1.825131240668002


In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(svm.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(svm.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(svm.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,svm.predict(X)))

In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(svm.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(svm.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(svm.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,svm.predict(X_test)))

## Modelo 7: *Naybe Bayes*

In [ ]:
%%time
#We create the Pipeline. First step, Normalize data, include the Polynomimal 
#(include bias false to remove from the Polynomial for X[0]=1) and then use the Ridge method
gnb_pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('gnb', GaussianNB())
])

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
#dists = {'gnb__alpha' : uniform(loc=0, scale=200)}
#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
#ridge_CV = RandomizedSearchCV(ridge_pipe, param_distributions=dists, n_iter=30, cv=5)
#Now we train the model
gnb_pipe.fit(X, y)

CPU times: user 109 ms, sys: 1.94 ms, total: 111 ms
Wall time: 113 ms


In [ ]:
print("The best X,y score is: ", gnb_pipe.score(X, y))
print("The best X,y test score is: ", gnb_pipe.score(X_test, y_test))
print("The MAE X,y is: ", mean_absolute_error(gnb_pipe.predict(X), y))
print("The MAE test X,y is: ", mean_absolute_error(y_test,gnb_pipe.predict(X_test)))

The best X,y score is:  0.006657860916749964
The best X,y test score is:  0.005997429672997287
The MAE X,y is:  10.130515493359988
The MAE test X,y is:  10.173639868627731


In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(gnb_pipe.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(gnb_pipe.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(gnb_pipe.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,gnb_pipe.predict(X)))

Error absoluto medio (MAE): 10.130515493359988
Error Cuadrático Medio (MSE): 116.98725546194488
Raíz del Error Cuadrático Medio (RMSE): 10.816064693868324
Coeficiente de determinación R2: -10.78138182835229


In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(gnb_pipe.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(gnb_pipe.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(gnb_pipe.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,gnb_pipe.predict(X_test)))

Error absoluto medio (MAE): 10.173639868627731
Error Cuadrático Medio (MSE): 117.79251749250321
Raíz del Error Cuadrático Medio (RMSE): 10.853226132929471
Coeficiente de determinación R2: -10.824912069631436


## Modelo 8: *Support Vector Machines, SVM con RandomizedSearch*

In [ ]:
%%time
#We create the Pipeline. First step, Normalize data, include the Polynomimal 
#(include bias false to remove from the Polynomial for X[0]=1) and then use the Ridge method
svr_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVR(C=1000, epsilon=0.5))
])

#In a dictionary, Step name _ _ and then parameter name, and then : with the type to do
dists = {'svm__C' : [1000, 3000, 5000], 'svm__epsilon': [0.001, 0.1, 1]}
#Cross Validation (CV uses the n parts (1/5) for cv=5, n_inter is the number of models, )
svr_CV = RandomizedSearchCV(svr_pipe, param_distributions=dists, n_iter=3, cv=5)
#Now we train the model
svr_CV.fit(X, y)

Wall time: 8h 36s


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('svm',
                                              SVR(C=1000, epsilon=0.5))]),
                   n_iter=3,
                   param_distributions={'svm__C': [1000, 3000, 5000],
                                        'svm__epsilon': [0.001, 0.1, 1]})

In [ ]:
print("The best Parameters are :", svr_CV.best_params_)
print("The best X,y score is: ", svr_CV.score(X, y))
print("The best X,y test score is: ", svr_CV.score(X_test, y_test))
print("The MAE X,y is: ", mean_absolute_error(svr_CV.predict(X), y))
print("The MAE test X,y is: ", mean_absolute_error(svr_CV.predict(X_test), y_test))

The best Parameters are : {'svm__epsilon': 1, 'svm__C': 1000}
The best X,y score is:  0.48312998658187045
The best X,y test score is:  0.45485682437869546
The MAE X,y is:  1.7693013134400775
The MAE test X,y is:  1.8224817093740744


In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(svr_CV.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(svr_CV.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(svr_CV.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,svm.predict(X)))

In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(svr_CV.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(svr_CV.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(svr_CV.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,svr_CV.predict(X_test)))

## Modelo 9: *Random Forest*

**Se utiliza un cross validation para encontrar los mejores hiperparámetros para la función RandomForestRegresor**

In [12]:
%%time
param_grid = {'n_estimators': [50,100,150],
              'min_samples_leaf': [2, 3, 5],
              'max_depth'   : [None,1, 3, 10, 20]
             }
grid = GridSearchCV(
        estimator  = RandomForestRegressor(random_state = 123),
        param_grid = param_grid,
       )

grid.fit(X = X, y = y)

CPU times: user 1h 3min 8s, sys: 3.97 s, total: 1h 3min 12s
Wall time: 1h 2min 59s


In [13]:
print("Mejores hiperparámetros encontrados (cv)")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

Mejores hiperparámetros encontrados (cv)
{'max_depth': 10, 'min_samples_leaf': 2, 'n_estimators': 150} : 0.46055530868663 None


In [14]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(grid.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(grid.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(grid.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,grid.predict(X)))

Error absoluto medio (MAE): 1.7409488831046758
Error Cuadrático Medio (MSE): 4.870506568666651
Raíz del Error Cuadrático Medio (RMSE): 2.206922420173997
Coeficiente de determinación R2: 0.5095081309807676


In [15]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(grid.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(grid.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(grid.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,grid.predict(X_test)))

Error absoluto medio (MAE): 1.824030968341125
Error Cuadrático Medio (MSE): 5.355657435106631
Raíz del Error Cuadrático Medio (RMSE): 2.314229339349631
Coeficiente de determinación R2: 0.4623582245006834


**Se entrena un segundo cross validation acotando los resultados para encontrar los mejores hiperparámetros para la función RandomForestRegresor** 

In [16]:
param_grid = {'n_estimators': [150,200],
              'min_samples_leaf': [1,3,5,7],
              'max_depth'   : [13,15,20]
             }
grid2 = GridSearchCV(
        estimator  = RandomForestRegressor(random_state = 123),
        param_grid = param_grid,
       )

grid2.fit(X = X, y = y)

KeyboardInterrupt: ignored

In [ ]:
print("Mejores hiperparámetros encontrados (cv)")
print(grid2.best_params_, ":", grid2.best_score_, grid2.scoring)

Mejores hiperparámetros encontrados (cv)


NameError: ignored

In [ ]:
#Medidas de entrenamiento
print('Error absoluto medio (MAE):',mean_absolute_error(grid2.predict(X), y))
print('Error Cuadrático Medio (MSE):',mean_squared_error(grid2.predict(X), y))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(grid2.predict(X), y)))
print('Coeficiente de determinación R2:',r2_score(y,grid2.predict(X)))

In [ ]:
#Medidas en la prueba
print('Error absoluto medio (MAE):',mean_absolute_error(grid2.predict(X_test), y_test))
print('Error Cuadrático Medio (MSE):',mean_squared_error(grid2.predict(X_test), y_test))
print('Raíz del Error Cuadrático Medio (RMSE):',np.sqrt(mean_squared_error(grid2.predict(X_test), y_test)))
print('Coeficiente de determinación R2:',r2_score(y_test,grid2.predict(X_test)))